In [1]:
# ====================
# 🔹 STEP 1: Install Required Packages
# ====================
!pip install -q transformers datasets peft accelerate bitsandbytes sentencepiece

# ====================
# 🔹 STEP 2: Load Base Model (Phi-2 or Mistral)
# ====================
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "microsoft/phi-2"  # Use "mistralai/Mistral-7B-v0.1" if you have a GPU
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

# ====================
# 🔹 STEP 3: Prepare Dataset (Muslim Family Law - Pakistan)
# ====================
from datasets import Dataset

data = [
    {
        "instruction": "What are the rights of a wife under Muslim Family Law in Pakistan?",
        "output": "Under the Muslim Family Laws Ordinance 1961, a wife has rights including maintenance, dower, and divorce under certain conditions..."
    },
    {
        "instruction": "Draft a simple divorce notice under Pakistani law.",
        "output": "Subject: Notice of Divorce\n\nTo: [Wife's Name],\n\nI, [Husband's Name], hereby pronounce divorce (Talaq)..."
    },
    {
        "instruction": "Explain the concept of 'Khula' in Pakistan.",
        "output": "Khula is a form of divorce initiated by the wife, typically through the Family Court, when she cannot live with her husband..."
    }
]

dataset = Dataset.from_list(data)

# ====================
# 🔹 STEP 4: Format Dataset for Instruction Tuning
# ====================
def format_sample(sample):
    return {
        "text": f"### Instruction:\n{sample['instruction']}\n\n### Response:\n{sample['output']}"
    }

dataset = dataset.map(format_sample)

# ====================
# 🔹 STEP 5: Apply LoRA for Parameter-Efficient Fine-Tuning
# ====================
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # May vary by model
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ====================
# 🔹 STEP 6: Train the Model
# ====================
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_steps=10,
    output_dir="./lawyer_bot_model",
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

# ====================
# 🔹 STEP 7: Save the Fine-tuned Model
# ====================
model.save_pretrained("./lawyer_bot_model")
tokenizer.save_pretrained("./lawyer_bot_model")

# ====================
# 🔹 STEP 8: Use the Lawyer Bot
# ====================
from transformers import pipeline

pipe = pipeline("text-generation", model="./lawyer_bot_model", tokenizer="./lawyer_bot_model")

response = pipe("### Instruction:\nWhat is the procedure of Khula in Pakistan?\n\n### Response:", max_new_tokens=200)[0]['generated_text']
print(response)


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      Traceback (most recent call last):
        File "C:\Users\hisham.fawad\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "C:\Users\hisham.fawad\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\hisham.fawad\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\hisham.fawad\AppData\Local\Temp\pip-build-env-35o

ModuleNotFoundError: No module named 'transformers'